In [0]:

%pip install typing-extensions
%pip install openai
%pip install sqlparse==0.5.0
%pip install mlflow>=2.9.0
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 901.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk

### Trying to register to MLFlow

In [0]:
%pip install mlflow[databricks]
import mlflow

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached google_cloud_storage-2.16.0-py2.py3-none-any.whl (125 kB)
  Using cached azure_storage_file_datalake-12.15.0-py3-none-any.whl (254 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached azure_core-1.30.2-py3-none-any.whl (194 kB)
  Using cached azure_storage_blob-12.20.0-py3-none-any.whl (392 kB)
  Using cached google_auth-2.30.0-py2.py3-none-any.whl (193 kB)
  Using cached google_api_core-2.19.0-py3-none-any.whl (139 kB)
  Using cached google_crc32c-1.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (32 kB)
  Using cached google_resumable_media-2.7.0-py2.py3-none-any.whl (80 kB)
  Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl (29 kB)
  Using cached proto_plus-1.23.0-py3-none-any.whl (48 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached pyasn1_modules-0.4.0-py3-none-any.whl (181 kB)
  Using cached pyasn1-

In [0]:
# Import the necessary libraries
import mlflow
from mlflow.pyfunc import PythonModel, ModelSignature
from mlflow.types import DataType, Schema
import mlflow.pyfunc
import mlflow.deployments
import os

def get_table_schema(table_name, spark_object):
    table_schema = spark_object.sql("DESCRIBE {}".format(table_name))
    my_schema = table_schema.collect()
    return (table_name, my_schema)

In [0]:

# Define the custom model class for TxtToSQL
class TxtToSQLModel(PythonModel):
    # Constructor
    def __init__(self):
        pass

    def nl_to_sql(self, client, nl_query, schema):
        chat_response = client.predict(
            endpoint="databricks-meta-llama-3-70b-instruct",
            inputs={
                "messages": [
                    {"role": "system", "content": "You are an AI assistant"},
                    {
                        "role": "user",
                        "content": f"Table schema:\n{schema}\n\nConvert the following natural language query to SQL: {nl_query}\n\nSQL: and in the output give only the SQL query without text",
                    },
                ],
                "temperature": 0.1,
                "max_tokens": 256,
            },
        )

        return chat_response.choices[0]

    # Method for loading the model
    def load_context(self, context):
        pass

    """
    # Method for predicting using the loaded model
    def predict(self, model_input):
        # Custom prediction logic goes here
        pass
    """
    def predict(self, client, message, table_name, spark_object):
        table_schema = get_table_schema(table_name, spark_object)
        sql_query = self.nl_to_sql(client, message, table_schema)
        response = sql_query["message"]["content"]
        response = response.replace("```", "")
        #df = spark_object.sql(response)
        return response

# Create an instance of the custom model
txtToSQLModel = TxtToSQLModel()

# Model Signature to be added to MLlow registration
from mlflow.types import DataType, Schema, ColSpec
input_schema = Schema([
    ColSpec("string", "message"),
    ColSpec("string", "table_name")
])
output_schema = Schema([
    ColSpec("string", "sql_query")
])
model_signature = ModelSignature(
    inputs=input_schema,
    outputs=output_schema
)

# # Log the model with MLflow
# mlflow.pyfunc.log_model(
#     "my_custom_model",
#     python_model=txtToSQLModel,
#     artifacts={},
#     signature=model_signature,
#     # registered_model_name="workspace.default.txt_to_sql_llama3",
# )

In [0]:

# Define the custom model class for TxtToSQL
class DefineTxtQuery(PythonModel):
    # Constructor
    def __init__(self):
        pass

    def nl_to_sql(self, client, nl_query, schema):
        chat_response = client.predict(
            endpoint="databricks-meta-llama-3-70b-instruct",
            inputs={
                "messages": [
                    {"role": "system", "content": "You are an AI assistant"},
                    {
                        "role": "user",
                        "content": f"From the table having schema :\n{schema}\n\nWhat would you look at to answer this question : {nl_query}\n\n Give an answer that contains simple statements looking like natural language queries",
                    },
                ],
                "temperature": 0.1,
                "max_tokens": 256,
            },
        )

        return chat_response.choices[0]

    # Method for loading the model
    def load_context(self, context):
        pass

    """
    # Method for predicting using the loaded model
    def predict(self, model_input):
        # Custom prediction logic goes here
        pass
    """
    def predict(self, client, message, table_name, spark_object):
        table_schema = get_table_schema(table_name, spark_object)
        sql_query = self.nl_to_sql(client, message, table_schema)
        response = sql_query["message"]["content"]
        response = response.replace("```", "")
        #df = spark_object.sql(response)
        return response

# Create an instance of the custom model
defineTxtQueryModel = DefineTxtQuery()

# Model Signature to be added to MLlow registration
from mlflow.types import DataType, Schema, ColSpec
input_schema = Schema([
    ColSpec("string", "message"),
    ColSpec("string", "table_name")
])
output_schema = Schema([
    ColSpec("string", "sql_query")
])
model_signature = ModelSignature(
    inputs=input_schema,
    outputs=output_schema
)

# # Log the model with MLflow
# mlflow.pyfunc.log_model(
#     "my_custom_model",
#     python_model=defineTxtQueryModel,
#     artifacts={},
#     signature=model_signature,
#     registered_model_name="workspace.default.define_txt_query_llama",
# )

### Testing the Model 


In [0]:
client = mlflow.deployments.get_deploy_client("databricks")

## 1st step ask a generic question, an agent will define how to solve this with the data that we have at hand

In [0]:
query = defineTxtQueryModel.predict(client, "What is the cheapest place to buy a house past the date timestamp year 2023 for a familiy having more than 2 bathrooms? Always Keep the Latitude and Longitude columns.", "bright_data_real_estate_listings.datasets.zillow_properties", spark)

print(query)

To answer this question, I would look at the following columns:

* TIMESTAMP: to filter for dates past 2023
* BATHROOMS: to filter for houses with more than 2 bathrooms
* PRICE: to find the cheapest options
* LATITUDE and LONGITUDE: to keep these columns in the result

Here's a natural language query that summarizes the requirements:

"Show me the houses with more than 2 bathrooms, listed after 2023, sorted by price from lowest to highest, and include their latitude and longitude."

In SQL, this would translate to:
sql
SELECT *
FROM bright_data_real_estate_listings.datasets.zillow_properties
WHERE TIMESTAMP >= '2023-01-01' 
  AND BATHROOMS > 2 
  AND PRICE IS NOT NULL 
ORDER BY PRICE ASC;

Note that I added `PRICE IS NOT NULL` to ensure we only consider houses with a valid price.


## 2nd step get you answer transformed into sql 

In [0]:

result = txtToSQLModel.predict(client, query, "bright_data_real_estate_listings.datasets.zillow_properties", spark)

print(result)


SELECT *
FROM bright_data_real_estate_listings.datasets.zillow_properties
WHERE TIMESTAMP >= '2023-01-01' 
  AND BATHROOMS > 2 
  AND PRICE IS NOT NULL 
ORDER BY PRICE ASC;


## 3rd step : get the query executed 

In [0]:
from pyspark.sql import functions as F
final_df = spark.sql(result)

final_display = final_df.groupBy("City").agg(F.mean('LATITUDE').cast('DOUBLE').alias('LATITUDE'), F.mean('LONGITUDE').alias('LONGITUDE'), F.mean('Price').cast('DOUBLE').alias('PRICE'))

df_plot = final_display.toPandas()

display(df_plot)


City,LATITUDE,LONGITUDE,PRICE
Tyler,32.32733,-95.192955,0.0
South Egremont,42.141712,-73.41186,995000.0
Harrisburg,40.339207,-76.73647,348990.0
Springfield,37.66069,-85.25994,899900.0
Corona,33.963272,-117.60413,990000.0
Lawrenceville,33.982054,-84.02543750000001,421000.0
Wellington,26.640866666666668,-80.23280466666667,1233400.0
North Las Vegas,36.30251,-115.16812,566340.0
Auburn,34.056335,-83.799805,1699000.0
Harrison,40.9881,-73.71914,2995000.0


In [0]:
import plotly.express as px

# Example dataframe for plotting
# df_plot = final_display.toPandas()

fig = px.scatter_mapbox(df_plot, lat="LATITUDE", lon="LONGITUDE", color="PRICE",
                        size="PRICE", color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max=20, zoom=2.5, mapbox_style="carto-positron")

# Set title
fig.update_layout(title="Real Estate Prices")

# Control dimensions
fig.update_layout(
    title="Real Estate Prices",
    title_x=0.5,  # Center title
    title_font_size=24,  # Change title size
    title_font_color="white",  # Update title text color
    paper_bgcolor="black",  # Set background color
    legend_font_color="white",  # Set legend text color to white
    font_color="white"  # Set all font color to white
)

# Rename the display name of a column in the figure
fig.update_traces(marker=dict(symbol='circle', opacity=0.8), name="Average Price")

# Control dimensions
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

fig.show()